In [16]:
%pip install pyarrow

     ---------------------------------------- 25.3/25.3 MB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Romain\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
import csv
import re
import polars as pl

#### Project CSV

In [178]:
input_file = r"C:\Users\Romain\OneDrive - KU Leuven\Masters\MBIS\Year 2\Semester 2\Modern Data Analytics\CORDIS\project.csv"
output_file = r"C:\Users\Romain\OneDrive - KU Leuven\Masters\MBIS\Year 2\Semester 2\Modern Data Analytics\CORDIS\project_cleaned.csv"


year_pattern = re.compile(r"^20\d{2}")

DELIM_IN = ";"
DELIM_OUT = ";"

with open(input_file, "r", encoding="utf-8") as f_in, \
     open(output_file, "w", encoding="utf-8", newline="") as f_out:
    
    # Use the chosen output delimiter
    writer = csv.writer(f_out, delimiter=DELIM_OUT)

    # Keep track if we're on the header row (just once)
    header = True

    for j,line in enumerate(f_in):
        # Split by the *input* delimiter
        fields = line.rstrip("\n").split(DELIM_IN)

        cleaned_fields = []
        descr = True
        bad_field = fields
        for i, field in enumerate(fields):
            # If the first field is blank, skip row
            if i == 0 and field == '':
                break
            if field[0]!='"' and not header and not year_pattern.match(field):
                field = field.replace('"""', '').replace('""', '').replace('"', '')
                field = field.strip('"').strip()
                if cleaned_fields:
                    cleaned_fields[-1] += " " + field
                else:
                    cleaned_fields.append(field)
            elif field == '' and i > 20:
                continue
            else:
                field = field.replace('"""', '').replace('""', '').replace('"', '')
                field = field.strip('"').strip()
                cleaned_fields.append(field)
        header = False
        while cleaned_fields and not cleaned_fields[-1].strip():
            cleaned_fields.pop()
        if cleaned_fields[0] in ['101057843','101046298']:
            last_three = cleaned_fields[-3:] 
            cleaned_fields[-4], cleaned_fields[-3], cleaned_fields[-2] = last_three
            cleaned_fields.pop()
            
        if cleaned_fields:
            cleaned_fields[-1] = cleaned_fields[-1].replace(",", "")

        # Write the final row
        if cleaned_fields:  # only write if there's something
            writer.writerow(cleaned_fields)

projects = pl.read_csv(output_file,separator=";",quote_char=None,has_header=True,infer_schema_length=40000,truncate_ragged_lines=True)
projects = projects.with_columns([
    # 1) ID as Int64
    pl.col("id").cast(pl.Int64),
    # 2) Remove extra quotes from string columns that should remain strings
    pl.col("acronym").str.strip_chars('"').alias("acronym"),
    pl.col("status").str.strip_chars('"').alias("status"),
    pl.col("title").str.strip_chars('"').alias("title"),
    pl.col("legalBasis").str.strip_chars('"').alias("legalBasis"),
    pl.col("topics").str.strip_chars('"').alias("topics"),
    pl.col("frameworkProgramme").str.strip_chars('"').alias("frameworkProgramme"),
    pl.col("masterCall").str.strip_chars('"').alias("masterCall"),
    pl.col("subCall").str.strip_chars('"').alias("subCall"),
    pl.col("fundingScheme").str.strip_chars('"').alias("fundingScheme"),
    pl.col("nature").str.strip_chars('"').alias("nature"),
    pl.col("objective").str.strip_chars('"').alias("objective"),
    pl.col("grantDoi").str.strip_chars('"').alias("grantDoi"),
    
    # 3) Convert totalCost and ecMaxContribution to floats (or Int64 if you prefer)
    pl.col("totalCost").str.strip_chars('"').str.replace(",",".").cast(pl.Float64).alias("totalCost"),
    pl.col("ecMaxContribution").str.strip_chars('"').str.replace(",",".").cast(pl.Float64).alias("ecMaxContribution"),
    
    # 4) Convert date-like columns from string to Polars date or datetime
    # If they have a date format like YYYY-MM-DD, use pl.Date
    # If they also have times, use pl.Datetime
    pl.col("startDate")
      .str.strip_chars('"')
      .str.strptime(pl.Date, "%Y-%m-%d", strict=False)
      .alias("startDate"),
    
    pl.col("endDate")
      .str.strip_chars('"')
      .str.strptime(pl.Date, "%Y-%m-%d", strict=False)
      .alias("endDate"),
    
    pl.col("ecSignatureDate")
      .str.strip_chars('"')
      .str.strptime(pl.Date, "%Y-%m-%d", strict=False)
      .alias("ecSignatureDate"),
    
    pl.col("contentUpdateDate")
      .str.strip_chars('"')
      .str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S", strict=False)
      .alias("contentUpdateDate"),
    
    # 5) Convert rcn to Int64
    pl.col("rcn").cast(pl.Int64).alias("rcn"),
])
projects.write_parquet("projects.parquet")

In [179]:
projects = pl.read_parquet("projects.parquet")
projects.head()

id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,ecSignatureDate,frameworkProgramme,masterCall,subCall,fundingScheme,nature,objective,contentUpdateDate,rcn,grantDoi
i64,str,str,str,date,date,f64,f64,str,str,date,str,str,str,str,str,str,datetime[μs],i64,str
101116741,"""DOE""","""SIGNED""","""Digitizing Other Economies: A …",2024-02-01,2029-01-31,1.499998e6,1.499998e6,"""HORIZON.1.1""","""ERC-2023-STG""",2023-12-15,"""HORIZON""","""ERC-2023-STG""","""ERC-2023-STG""","""HORIZON-ERC""",null,"""How do longstanding, primarily…",2023-12-26 14:40:12,259247,"""10.3030/101116741"""
101163161,"""IRASTRO""","""SIGNED""","""MOLECULAR QUANTUM DYNAMICS IN …",2025-08-01,2031-07-31,1.2085363e7,1.2085363e7,"""HORIZON.1.1""","""ERC-2024-SyG""",2025-02-20,"""HORIZON""","""ERC-2024-SyG""","""ERC-2024-SyG""","""HORIZON-ERC-SYG""",null,"""The James Webb Space Telescope…",2025-02-24 17:23:14,268970,"""10.3030/101163161"""
101160499,"""In-phase""","""SIGNED""","""Multiscale modelling of aberra…",2025-06-01,2030-05-31,1.489128e6,1.489128e6,"""HORIZON.1.1""","""ERC-2024-STG""",2025-02-19,"""HORIZON""","""ERC-2024-STG""","""ERC-2024-STG""","""HORIZON-ERC""",null,"""The spatiotemporal organizatio…",2025-02-24 17:23:11,268969,"""10.3030/101160499"""
101166905,"""mw-atlas""","""SIGNED""","""The first comprehensive Atlas …",2025-05-01,2031-04-30,9.95756e6,9.95756e6,"""HORIZON.1.1""","""ERC-2024-SyG""",2025-02-20,"""HORIZON""","""ERC-2024-SyG""","""ERC-2024-SyG""","""HORIZON-ERC-SYG""",null,"""The Milky Way is the cosmic en…",2025-02-24 17:23:01,268971,"""10.3030/101166905"""
101162875,"""MAtCHLESS""","""SIGNED""","""Untapping multiparametric 2D l…",2025-03-01,2030-02-28,1.5e6,1.5e6,"""HORIZON.1.1""","""ERC-2024-STG""",2025-02-18,"""HORIZON""","""ERC-2024-STG""","""ERC-2024-STG""","""HORIZON-ERC""",null,"""Cellular organisms are complex…",2025-02-24 17:23:12,268966,"""10.3030/101162875"""


#### Project Deliverables

In [102]:
input_file = r"C:\Users\Romain\OneDrive - KU Leuven\Masters\MBIS\Year 2\Semester 2\Modern Data Analytics\CORDIS\projectDeliverables.csv"
output_file = r"C:\Users\Romain\OneDrive - KU Leuven\Masters\MBIS\Year 2\Semester 2\Modern Data Analytics\CORDIS\projectDeliverables_cleaned.csv"


number_pattern = re.compile(r"\d{4}")

DELIM_IN = ";"
DELIM_OUT = ";"

with open(input_file, "r", encoding="utf-8") as f_in, \
     open(output_file, "w", encoding="utf-8", newline="") as f_out:
    
    writer = csv.writer(f_out, delimiter=DELIM_OUT)

    header = True

    for j, line in enumerate(f_in):
        fields = line.rstrip("\n").split(DELIM_IN)
        total_fields=[]
        for i, field in enumerate(fields):
            field = field.replace('"','')
            total_fields.append(field)
        writer.writerow(total_fields)

projects = pl.read_csv(output_file,separator=";",quote_char=None,has_header=True,infer_schema_length=40000,truncate_ragged_lines=True)

In [103]:
projects.head()

id,title,deliverableType,description,projectID,projectAcronym,url,collection,contentUpdateDate,rcn
str,str,str,str,str,str,str,str,str,str
"""190136266_5_DELIVHORIZON""","""Project webpage and logo""","""Websites, patent fillings, vid…","""Project webpage and logo""","""190136266""","""Deepstroke-Acc""","""https://ec.europa.eu/research/…","""Project deliverable""","""2024-01-31 13:50:03""","""1038413"""
"""190177099_5_DELIVHORIZON""","""White paper on sustainable spa…","""Documents, reports""","""White paper on sustainable spa…","""190177099""","""Aurora Plasma Brake (APB)""","""https://ec.europa.eu/research/…","""Project deliverable""","""2024-03-28 13:26:00""","""1067360"""
"""101129658_11_DELIVHORIZON""","""Dissemination and Communicatio…","""Documents, reports""","""Planning and reporting of comp…","""101129658""","""SPATRA""","""https://ec.europa.eu/research/…","""Project deliverable""","""2024-05-06 11:02:10""","""1090541"""
"""101129658_4_DELIVHORIZON""","""Requirements and use cases spe…","""Documents, reports""","""Report on defined requirements…","""101129658""","""SPATRA""","""https://ec.europa.eu/research/…","""Project deliverable""","""2024-12-02 16:52:21""","""1213967"""
"""101129658_2_DELIVHORIZON""","""Risk management and project qu…","""Documents, reports""","""Risk management and Quality Pl…","""101129658""","""SPATRA""","""https://ec.europa.eu/research/…","""Project deliverable""","""2024-04-16 15:36:09""","""1079345"""
